In [1]:
import matplotlib.pyplot as plt
from lr_steps import *
import matplotlib.pyplot as plt
import pickle
import time

In [2]:
def run_lr_algo(d, r, b, sigma,
                lr_r, lr_f, beta_r, beta_f,
                m, avg_frac, stop_frac, T_r, T_f,
                times):
    x = np.random.randn(b, d, 1) * sigma
    regrets = np.zeros([times, m+1])
    for t in range(times):
        start_time = time.time()
        print(f"#Exp={t}")
        R, f, p, o, regrets_t = algorithm(x=x, r=r,
                                       lr_r=lr_r, lr_f=lr_f, beta1=beta_r, beta2=beta_f,
                                       m=m, T_r=T_r, T_f=T_f, avg_frac=avg_frac, stop_frac=stop_frac)
        regrets[t, :] = regrets_t
        print(regrets_t[-1])
        run_time_seconds = time.time() - start_time
        print(f"Running Time: {run_time_seconds // 60}:{run_time_seconds % 60}")

    return regrets

def plot_and_save(x_axis, mean_regrets, std_regrets,
          label_char, labels_val, x_label, y_label, title, name):
    plt.grid()
    for i in range(mean_regrets.shape[0]):
        plt.plot(x_axis, mean_regrets[i], label=label_char+f"={labels_val[i]}")
        plt.fill_between(x_axis, mean_regrets[i]-std_regrets[i], mean_regrets[i]+std_regrets[i],
                         color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    plt.savefig(f"{name}.png")
    plt.clf()

    pickle.dump(mean_regrets, open(f'{name}_mean.p', 'wb'))
    pickle.dump(std_regrets, open(f'{name}_std.p', 'wb'))


In [3]:
b = 100
d = 15
r = 3
sigma=1

lr_r = 1e-3
lr_f = 1e-1
beta_r = 0.9
beta_f = 0.9

m = 10
avg_frac=4
stop_frac=4
T_r=10
T_f=10

times = 3

In [4]:
regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                      lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                      m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                      times=times)

plt.grid()
plt.title("logistic regression regret vs iteration")
plt.xlabel("#Iteration")
plt.ylabel("Regret")
plt.plot(range(m+1), regrets.T.mean(axis=1), label=r"$d$="+f"{d}")
plt.fill_between(range(m+1),
                 regrets.T.mean(axis=1)+regrets.T.std(axis=1), regrets.T.mean(axis=1)-regrets.T.std(axis=1),
                 color='r', alpha=0.5)
plt.legend()
name = f"log-reg_reg-vs-iter_d={d}_r={r}_b={b}"
plt.savefig(f"{name}.png")
pickle.dump(regrets, open(f'{name}.p', 'wb'))
plt.clf()

#Exp=0
0.08994405973824728
Running Time: 0.0:2.446608781814575
#Exp=1
0.08978384439912707
Running Time: 0.0:2.2912964820861816
#Exp=2
0.08866565417327171
Running Time: 0.0:1.969059944152832


<Figure size 432x288 with 0 Axes>

In [5]:
max_m = 10
m_step = 3

d_start = 5
d_max = 20
d_step = 3

m_list = list(range(1, max_m+1, m_step))
d_list = list(range(d_start, d_max+1, d_step))

mean_regrets = np.zeros([len(m_list), len(d_list)])
std_regrets = np.zeros([len(m_list), len(d_list)])
for m_idx, m in enumerate(m_list):
    for d_idx, d in enumerate(d_list):
        print(f"*** m={m}, d={d} ***")
        regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                              lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                              m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                              times=times)
        mean_regrets[m_idx, d_idx] = regrets.min(axis=1).mean()
        std_regrets[m_idx, d_idx] = regrets.min(axis=1).std()


plot_and_save(x_axis=d_list, mean_regrets=mean_regrets, std_regrets=std_regrets,
              label_char=r"$m$", labels_val=m_list, x_label=r'$d$', y_label=f'Regret',
              title=r"logistic regression regret vs $d$"+f"_r={r}",
              name=f"reg-vs-d_r={r}_b={b}")

*** m=1, d=5 ***
#Exp=0
0.09128744403844671
Running Time: 0.0:0.6778006553649902
#Exp=1
0.07144254649910353
Running Time: 0.0:0.46689319610595703
#Exp=2
0.06166595041464175
Running Time: 0.0:0.5819997787475586
*** m=1, d=8 ***
#Exp=0
0.09566766847429714
Running Time: 0.0:0.10300016403198242
#Exp=1
0.07572298913402514
Running Time: 0.0:0.13100051879882812
#Exp=2
0.06272480135906044
Running Time: 0.0:0.09006810188293457
*** m=1, d=11 ***
#Exp=0
0.06989797937389772
Running Time: 0.0:0.07800054550170898
#Exp=1
0.0778470006582702
Running Time: 0.0:0.07500004768371582
#Exp=2
0.07864730217379669
Running Time: 0.0:0.10604310035705566
*** m=1, d=14 ***
#Exp=0
0.09215045692100272
Running Time: 0.0:0.09143185615539551
#Exp=1
0.08889188928934152
Running Time: 0.0:0.045424699783325195
#Exp=2
0.09599994678230134
Running Time: 0.0:0.05826926231384277
*** m=1, d=17 ***
#Exp=0
0.0953763408532643
Running Time: 0.0:0.04900479316711426
#Exp=1
0.09340600850577108
Running Time: 0.0:0.03899526596069336
#Exp=

<Figure size 432x288 with 0 Axes>

In [6]:
max_m = 10
m_step = 3

s_start = 0.5
s_max = 3
s_step = 0.5

m_list = list(range(1, max_m+1, m_step))
s_list = np.linspace(s_start, s_max, int((s_max-s_start)/s_step)+1)

mean_regrets = np.zeros([len(m_list), len(d_list)])
std_regrets = np.zeros([len(m_list), len(d_list)])
for m_idx, m in enumerate(m_list):
    for s_idx, sigma in enumerate(s_list):
        print(f"*** m={m}, sigma={sigma} ***")
        regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                              lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                              m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                              times=times)
        mean_regrets[m_idx, s_idx] = regrets.min(axis=1).mean()
        std_regrets[m_idx, s_idx] = regrets.min(axis=1).std()


plot_and_save(x_axis=s_list, mean_regrets=mean_regrets, std_regrets=std_regrets,
              label_char=r"$m$", labels_val=m_list, x_label=r'$\sigma_0^2$', y_label=f'Regret',
              title=r"logistic regression regret vs $\sigma_0^2$"+f"_r={r}",
              name=f"reg-vs-sigma_r={r}_b={b}")

*** m=1, sigma=0.5 ***
#Exp=0
0.025978810678389138
Running Time: 0.0:0.12079334259033203
#Exp=1
0.0248608274734542
Running Time: 0.0:0.1029970645904541
#Exp=2
0.03241507470264625
Running Time: 0.0:0.13699984550476074
*** m=1, sigma=1.0 ***
#Exp=0
0.08865226170745553
Running Time: 0.0:0.03700613975524902
#Exp=1
0.09255778569356032
Running Time: 0.0:0.048993825912475586
#Exp=2
0.07670789733706784
Running Time: 0.0:0.0410001277923584
*** m=1, sigma=1.5 ***
#Exp=0
0.15724996191471394
Running Time: 0.0:0.03303933143615723
#Exp=1
0.14097784151562884
Running Time: 0.0:0.023120403289794922
#Exp=2
0.16527843689910607
Running Time: 0.0:0.025085926055908203
*** m=1, sigma=2.0 ***
#Exp=0
0.22562082317983667
Running Time: 0.0:10.661211490631104
#Exp=1
0.21580643929173
Running Time: 0.0:10.817399263381958
#Exp=2
0.26675820864536004
Running Time: 0.0:11.373589992523193
*** m=1, sigma=2.5 ***
#Exp=0
0.292167799302534
Running Time: 0.0:50.276121377944946
#Exp=1
0.2613414828018875
Running Time: 0.0:27.0

<Figure size 432x288 with 0 Axes>